Sveučilište u Zagrebu<br>
Fakultet elektrotehnike i računarstva

## Uvod u znanost o podacima


# Priprema i vizualizacija podataka

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import matplotlib.pyplot as plt

## Prikaz podataka o datotekama i člancima

In [3]:
import os
import re

files = os.listdir('reuters21578')

columns = ['FILENAME', 'NEWID', 'OLDID', 'LEWISSPLIT', 'CGISPLIT', 'TOPICS_ENUM', 'TEXT TYPE', 'DATE', 'TITLE', 'AUTHOR', 'DATELINE', 'TOPICS', 'PLACES', 'PEOPLE', 'ORGS', 'EXCHANGES', 'COMPANIES',  'BODY']
rows = []

for filename in os.listdir('reuters21578'):
    if filename.endswith(".sgm"):
        file_content = open(os.path.join("reuters21578", filename), mode = 'r', encoding='ascii', errors='ignore').read()
        articles = file_content.split("</REUTERS>")
        articles[0] = articles[0][articles[0].find("\n")::] ## izbacimo prvi red, onaj <!DOCSTYLE ...
        #print(articles[0])
        
        for article in articles:
            if "<REUTERS" in article:
                red = {}
                red['FILENAME'] = filename
                red['NEWID'] = re.findall('NEWID="([^"]*)"', article)[0]
                red['OLDID'] = re.findall('OLDID="([^"]*)"', article)[0]
                red['TOPICS_ENUM'] = re.findall('TOPICS="([^"]*)"', article)[0]
                red['TEXT TYPE'] = "".join(re.findall('TEXT TYPE="([^"]*)"', article))
                red['LEWISSPLIT'] = re.findall('LEWISSPLIT="([^"]*)"', article)[0]
                red['CGISPLIT'] = re.findall('CGISPLIT="([^"]*)"', article)[0]
                for znacajka in columns[7:]:
                    if znacajka == "DATE":
                        regex = '<' + znacajka + '>' + '([^&#$]*)' + '</' + znacajka + '>'
                    else:
                        regex = '<' + znacajka + '>' + '([\s\S]*)' + '</' + znacajka + '>'
                    match = re.findall(regex, article)
                    if(len(match)==0):
                        continue
                    elif match[0] == '':
                        continue
                    else:
                        match = match[0]
                        if '<D>' in match:
                            match = match.replace('<D>', '')
                            match = match.split('</D>')
                            match = match[:-1]
                        red[znacajka] = match
                #print(red[0:-1])
                rows.append(red.copy())

dataframe = pd.DataFrame(rows, columns=['NEWID', 'TOPICS', 'PLACES', 'PEOPLE', 'ORGS', 'EXCHANGES', 'TITLE', 'AUTHOR', 'DATELINE', 'OLDID', 'TOPICS_ENUM', 'FILENAME', 'DATE', 'COMPANIES', 'BODY', 'TEXT TYPE', 'LEWISSPLIT', 'CGISPLIT'])
dataframe = dataframe.astype('string')
dataframe = dataframe.astype({'NEWID' : 'int64', 'OLDID' : 'int64'})
#dataframe = dataframe.set_index("NEWID")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'reuters21578'

### Prikaz prvih 10 redova

In [ ]:
dataframe.head(n=10)

### Pohrana Dataframe-a

In [ ]:
filename = "clanci.csv"
dataframe.to_csv(filename)

In [ ]:
clanci = pd.read_csv(filename, index_col = 0)
clanci.head(n=10)

### Pregled podataka

In [ ]:
broj_primjeraka, broj_znacajki = clanci.shape
print("broj primjeraka (članaka), N = {}".format(broj_primjeraka))
print("broj značajki, n = {}".format(broj_znacajki))

In [ ]:
print("Koliko članaka ima u svakoj datoteci:")
print(clanci.groupby('FILENAME').size())

Značajke:
- **newid** - id koji članak ima u Reuters-21578 klasifikaciji
- **topics** - lista kategorija tema kojima članak pripada
- **places** - lista kategorija mjesta kojima članak pripada
- **people** - lista kategorija ljudi kojima članak pripada
- **orgs** - lista kategorija orgs kojima članak pripada
- **exchange** - lista kategorija razmjena kojima članak pripada
- **title** - naslov članka
- **author** - autor teksta
- **dateline** - od kod je članak potekao, i dan u godini
- **oldid** - id koji je članak imao u Reuters-22173 klasifikaciji
- **topics enum** - moguće vrijednosti su [YES, NO, BYPASS], je li dokument imao topics kategoriju u originalnom datasetu
- **filename** - ime datoteke u kojoj se nalazi članak
- **date** - datum i vrijeme dokumenta
- **companies** - ?
- **body** - tijelo teksta
- **text type** - moguće vrijednosti su [NORM, BRIEF, UNPROC], norm znači da tekst ima normalnu strukturu, brief znači da je tekst kratak, unproc znači da tekst ima neobičnu strukturu
- **lewissplit** - moguće vrijednosti su [TRAINING, TEST, NOT_USED] ovisno o tome za što se članak koristio u eksperimentima u izvješćima: LEWIS91d (poglavlja 9 i 10), LEWIS92b, LEWIS92e i LEWIS94b
- **cgisplit** - moguće vrijednosti su [TRAINING-SET, PUBLISHED-TESTSET] ovisno o tome za što se članak koristi u eksperimentima  u izvješćima HAYES89 i HAYES90b

In [ ]:
dataframe.info()

#### Monotoni atributi

In [ ]:
dataframe.nunique() # gledamo ako ima monotonih značajki

Zaključujemo da su NEWID i OLDID monotone značajke. 

In [ ]:
original = dataframe.copy() #čuvamo original podataka, just in case
podaci = original.copy()

podaci = podaci.drop('OLDID', axis=1, inplace=False)
podaci.drop('NEWID', axis=1, inplace=True)

In [ ]:
podaci.isna().sum() # broji nul vrijednosti za značajke

Vidimo da osim značajki za klasifikacijske oznake (topics, places, people, orgs, exchanges), značajke AUTHOR i COMPANIES imaju puno nul vrijednosti. Izbacujemo te značajke iz skupa podataka.

In [ ]:
podaci.drop(['AUTHOR', 'COMPANIES'], axis=1, inplace=True)

Izbacujemo zapise koji nemaju body jer nam oni zapravo ništa ne govore.

In [ ]:
podaci = podaci.loc[podaci.BODY.notnull(), :] # zadržavamo samo one zapise kojima body nije null

#### Label Encoder
Mijenjamo format značajki koje imaju malo unique vrijednosti (poprimaju vrijednosti iz nekog diskretnog skupa) - topics_enum, text type, lewisspit, cgisplit.

In [ ]:
le = LabelEncoder()
le.fit(podaci.loc[:, 'TOPICS_ENUM'])
podaci.loc[:, 'TOPICS_ENUM'] = le.transform(podaci.loc[:, 'TOPICS_ENUM'])
le.fit(podaci.loc[:, 'TEXT TYPE'])
podaci.loc[:, 'TEXT TYPE'] = le.transform(podaci.loc[:, 'TEXT TYPE'])
le.fit(podaci.loc[:, 'LEWISSPLIT'])
podaci.loc[:, 'LEWISSPLIT'] = le.transform(podaci.loc[:, 'LEWISSPLIT'])
le.fit(podaci.loc[:, 'CGISPLIT'])
podaci.loc[:, 'CGISPLIT'] = le.transform(podaci.loc[:, 'CGISPLIT'])
podaci

In [ ]:
podaci.describe(include='all')

### Vizualizacija podataka

#### Pie chart

In [ ]:
count_per_lewissplit = clanci.groupby('LEWISSPLIT').count().loc[:, 'NEWID'].values
labels = clanci.groupby('LEWISSPLIT').count().index.values
percent = (count_per_lewissplit/len(clanci))*100

# pokazuje postotak članaka koji su se koristi u kojim skupinama
plt.figure(figsize=(10, 3))
plt.pie(percent, labels=labels, autopct='%.2f')
plt.title("Udio podataka za treniranje i testiranje, LEWIS SPLIT")
plt.show()

#### Inkonzistentni podaci

In [ ]:
# inkonzistentni podaci
dateline_helper_dataframe = podaci.copy()
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.upper()
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('JANUARY', 'JAN')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.upper().str.replace('FEBRUARY', 'FEB')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('MARCH', 'MAR').str.replace('NARCH', 'MAR')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('APRIL', 'APR')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('JUNE', 'JUN')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('JULY', 'JUL')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('AUGUST', 'AUG')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('SEPTEMBER', 'SEP')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('OCTOBER', 'OCT')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('NOVEMBER', 'NOV')
dateline_helper_dataframe['DATELINE'] = dateline_helper_dataframe.DATELINE.str.replace('DECEMBER', 'DEC')


dateline_helper_dataframe = dateline_helper_dataframe.loc[dateline_helper_dataframe.DATELINE.notnull(), :]
dateline_helper_dataframe = dateline_helper_dataframe.DATELINE.str.extract(r',[\s]*(\w+)[\s]*[\d]*[\s]*-')
dateline_helper_dataframe = dateline_helper_dataframe.loc[dateline_helper_dataframe[0].notnull(), :]

In [ ]:
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
count_months = {}
for month in months:
    count_months[month] = 0
    
nul = 0
for month in dateline_helper_dataframe.to_numpy():
    try:
        count_months[month[0]] += 1
    except:
        nul += 1

#print(count_months)
plt.figure(figsize=(15, 5))
plt.plot(count_months.keys(), count_months.values())
plt.grid()
plt.title("Broj objavljenih članaka po mjesecima u 1987. godini")
plt.show()

### Podaci o kategorijama

In [ ]:
exchanges = []
orgs = []
people = []
places = []
topics = []


file_content = open(os.path.join("reuters21578", "all-exchanges-strings.lc.txt"), mode = 'r').read()
exchanges = file_content.split("\n")
file_content = open(os.path.join("reuters21578", "all-orgs-strings.lc.txt"), mode = 'r').read()
orgs = file_content.split("\n")
file_content = open(os.path.join("reuters21578", "all-people-strings.lc.txt"), mode = 'r').read()
people = file_content.split("\n")
file_content = open(os.path.join("reuters21578", "all-places-strings.lc.txt"), mode = 'r').read()
places = file_content.split("\n")
file_content = open(os.path.join("reuters21578", "all-topics-strings.lc.txt"), mode = 'r').read()
topics = file_content.split("\n")

#print("Sve moguće vrijednosti klase exchanges:\n", exchanges)
#print("Sve moguće vrijednosti klase orgs:\n", orgs)
#print("Sve moguće vrijednosti klase people:\n", people)
#print("Sve moguće vrijednosti klase places:\n", places)
#print("Sve moguće vrijednosti klase topics:\n", topics)


In [ ]:
most_freq_topic = {}
for topic in topics:
    most_freq_topic[topic] = dataframe.TOPICS.str.contains(topic).sum()

del most_freq_topic['']
topic_map_top_10 = dict(sorted(most_freq_topic .items(), key=lambda x: -x[1])[0:10]) #sortiramo od najčešćeg topica prema manje čestima

plt.figure(figsize=(15, 5))
plt.bar(topic_map_top_10.keys(), topic_map_top_10.values())
plt.title("10 ukupno najčešćih topic-a")
plt.grid()
plt.show()

count_topic_train = {}
count_topic_test = {}
for topic in topic_map_top_10.keys():
    count_topic_train[topic] = dataframe[dataframe.LEWISSPLIT == "TRAIN"].TOPICS.str.contains(topic).sum()
    count_topic_test[topic] =  dataframe[dataframe.LEWISSPLIT == "TEST"].TOPICS.str.contains(topic).sum()

plt.figure(figsize=(15, 5))
plt.grid()
plt.bar(topic_map_top_10.keys(), count_topic_train.values(), color='red', label='TRAIN')
plt.bar(topic_map_top_10.keys(), count_topic_test.values(), color='green', label='TEST', bottom=list(count_topic_train.values())) # bottom -> postavi vrijednost NA count_per_title_died
plt.title("Udio podataka s određenom klasom u train/test setu")
plt.legend(loc='best')

In [ ]:
most_freq_exchanges = {}
for exchange in exchanges:
    most_freq_exchanges[exchange] = dataframe.EXCHANGES.str.contains(exchange).sum()

del most_freq_exchanges['']
exchanges_map_top_10 = dict(sorted(most_freq_exchanges.items(), key=lambda x: -x[1])[0:10]) #sortiramo od najčešćeg topica prema manje čestima

plt.figure(figsize=(15, 5))
plt.bar(exchanges_map_top_10.keys(), exchanges_map_top_10.values())
plt.title("10 ukupno najčešćih exchange-a")
plt.grid()
plt.show()

In [ ]:
most_freq_people = {}
for p in people:
    most_freq_people[p] = dataframe.PEOPLE.str.contains(p).sum()

del most_freq_people['']
people_map_top_10 = dict(sorted(most_freq_people.items(), key=lambda x: -x[1])[0:10]) #sortiramo od najčešćeg topica prema manje čestima

plt.figure(figsize=(15, 5))
plt.bar(people_map_top_10.keys(), people_map_top_10.values())
plt.title("10 ukupno najčešćih people-a")
plt.grid()
plt.show()

In [ ]:
most_freq_places = {}
for p in places:
    most_freq_places[p] = dataframe.PEOPLE.str.contains(p).sum()

del most_freq_places['']
places_map_top_10 = dict(sorted(most_freq_places.items(), key=lambda x: -x[1])[0:10]) #sortiramo od najčešćeg topica prema manje čestima

plt.figure(figsize=(15, 5))
plt.bar(places_map_top_10.keys(), places_map_top_10.values())
plt.title("10 ukupno najčešćih places-a")
plt.grid()
plt.show()

In [ ]:
most_freq_orgs = {}
for o in orgs:
    most_freq_orgs[o] = dataframe.ORGS.str.contains(o).sum()

del most_freq_orgs['']
orgs_map_top_10 = dict(sorted(most_freq_orgs.items(), key=lambda x: -x[1])[0:10]) #sortiramo od najčešćeg topica prema manje čestima

plt.figure(figsize=(15, 5))
plt.bar(orgs_map_top_10.keys(), orgs_map_top_10.values())
plt.title("10 ukupno najčešćih orgs-a")
plt.grid()
plt.show()

#### Dodavanje stupca

In [ ]:
podaci_added = podaci.copy()
podaci_added['WORD_COUNT'] = podaci_added.apply(lambda row: len(row.BODY), axis=1)
podaci_added.head()

### Histogram

In [ ]:
interval = (podaci_added.WORD_COUNT.max() - podaci_added.WORD_COUNT.min())/10
bins = [podaci_added.WORD_COUNT.min()]
xticks = [podaci_added.WORD_COUNT.min()]
names = []
for i in range(1, 10):
    bins.append(int(podaci_added.WORD_COUNT.min()+interval*i))
    xticks.append(int(podaci_added.WORD_COUNT.min()+interval*i))
    names.append(" " + str(int(podaci_added.WORD_COUNT.min() + (i-1)*interval)) + " - " + str(int(podaci_added.WORD_COUNT.min() + i*interval)))

podaci_added['WORD_COUNT_RANGE'] = pd.cut(podaci_added['WORD_COUNT'], bins, labels=names)

plt.hist(podaci_added.WORD_COUNT, color='blue', alpha=0.5)
plt.xlabel("Broj riječi")
plt.ylabel("Broj članaka")
plt.xticks(xticks)
plt.xlim(xmax = 7000)
plt.show()

### Kutijasti graf

In [ ]:
plt.boxplot(podaci_added.WORD_COUNT) # kružići su outliersi, whiskers su min i max vrijednosti, a narančasta crta je srednja vrijednost
plt.show()

### Podaci o riječima u tijelima članaka

In [ ]:
#!pip install nltk
import nltk as nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
dataframe = podaci.loc[podaci.BODY.notnull(), :]

dataframe['BODY'] = dataframe['BODY'].replace('[\n\t0-9]',' ', regex=True) # uklanjamo sve prijelaze u novi red, tabulatore i brojeve iz tijela članaka

dataframe['BODY'] = dataframe['BODY'].replace('[\.\-"\#$%&!/,;]',' ', regex=True) #

dataframe['BODY'] = dataframe['BODY'].str.lower()

dataframe['BODY'] = dataframe['BODY'].replace('[\s]', ' ', regex=True)

dataframe['BODY'] = dataframe['BODY'].replace('[^a-zA-Z0-9\n \.]', '', regex=True)

#dataframe['BODY'] = dataframe['BODY'].replace('[^\x00-\x7f]', '', regex=True)

#dataframe['BODY'] = dataframe['BODY'].encode()

from collections import Counter
count = Counter()

exclude = set(stopwords.words('english'))
#print(exclude)

for index, row in dataframe.iterrows():
    body = []
    for w in words_per_row:
        if w not in exlude:
            body.append(w)
    row['BODY'] = body
    
words_per_row = word_tokenize(dataframe['BODY'].str)
for words in words_per_row:
    count[word] += 1

dataframe.to_csv("clanci_stripped.csv")

plt.figure(figsize=(15, 5))
plt.title("10 najčešćih riječi")
for item in count.most_common(10):
    plt.bar(item[0], item[1])
plt.show()